# Global functions for deep learning
---
This notebook contains functions that are used in the making of the deep learning models. 

In [14]:
%reset -f

In [15]:
SEED=100 # for reproducibility

In [16]:
import os

import numpy as np
np.random.seed(SEED)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tabulate import tabulate
from datetime import datetime
from matplotlib import rc

import keras
print("keras.__version__ =", keras.__version__ )
from keras import layers, regularizers, Input, Model
from keras.optimizers import Adam

import tensorflow as tf
tf.set_random_seed(SEED)
print("tf.__version__ =", tf.__version__)
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from functions import MAE, RMSE, batch_generator

keras.__version__ = 2.2.4
tf.__version__ = 1.12.0


In [26]:
# configure matplotlib params and plotting
## use seaborn as this gives nicer plots than the standard 
sns.set()
sns.set_context('paper')
sns.set_style('whitegrid', {'axes.grid': True, 'grid.linestyle': '--'})

rc('figure', figsize=(15,6))
rc('xtick', labelsize=12)
rc('ytick', labelsize=12)
rc('axes', labelsize=13, titlesize=14)
rc('legend', fontsize=14, handlelength=2)

### Functions for plotting and visualisation
The following functions are used to visualise and make plots of the deep learning models. 

In [27]:
def plot_history(history, savepath=None):
    """
    Plots the training and validation loss history of the model in the training phase. 
    """
    epochs = history.epoch

    train_mae = history.history['loss']
    val_mae = history.history['val_loss']

    plt.figure()
    plt.plot(epochs, train_mae, marker='o', markersize='3.0', label=r'Training loss', color="darkred")
    plt.plot(epochs, val_mae, marker='o', markersize='3.0', label=r'Validation loss', color="darkblue")  
    plt.xlabel(r'Epoch')
    plt.ylabel(r'MAE')
    plt.legend(frameon=True)
    if savepath is not None: 
        plt.savefig(savepath)
    plt.show()

In [28]:
def make_n_predictions(model, x_data, y_data, nruns=50):
    """
    Make n predictions with a specified trained model. 
    """
    x = np.expand_dims(x_data, axis=0)
    y_true = y_data
    
    preds_matr = np.zeros(shape=(nruns,y_true.shape[0],y_true.shape[1])) # (nruns, predictions, num_targets)
    err_matr = np.zeros(shape=(nruns, y_true.shape[1]))  # (nruns, num_targets) - one error per target per run

    for run in range(nruns):
        preds = model.predict(x)[0]
        err = MAE(y_true, preds, vector=True)
        
        preds_matr[run] = preds
        err_matr[run] = err
    
    pred_means = np.array([np.mean(preds_matr[:,:,i], axis=0) for i in range(y_data.shape[-1])]).T
    pred_stds = np.array([np.std(preds_matr[:,:,i], axis=0) for i in range(y_data.shape[-1])]).T
    
    return preds_matr, err_matr

In [29]:
def plot_multiple_predictions(preds_matr, y_true, time_vec, target_tags, 
                              start_idx=0, n_obs=200, plotCI=False, savepath=None):
    """
    Plots different realisations of the model, alternatively a 95% CI interval. 
    :param preds_matr: 3D matrix - The "predictions matrix" as obtained by the function make_n_predictions.
    :param y_true: 2D matrix - The true target values, matrix of all targets. 
    :param start_idx: Integer - The start index the plot should start at.
    :param plotCI: Boolean - If a CI interval should be plotted. 
    
    :return None: Will only show the obtained plot. 
    """
    pred_means = np.array([np.mean(preds_matr[:,:,i], axis=0) for i in range(y_true.shape[-1])]).T
    pred_stds = np.array([np.std(preds_matr[:,:,i], axis=0) for i in range(y_true.shape[-1])]).T
    nruns = preds_matr.shape[0]
    
    start_idx = start_idx if start_idx < len(y_true) else max(0,len(y_true)-n_obs) 
    end_idx = min(len(y_true),start_idx+n_obs)
    
    time = time_vec[start_idx:end_idx]
    
    for output in range(preds_matr.shape[-1]):
        
        plt.figure()
        
        if not plotCI: # then plot individual predictions 
            for run in range(nruns):
                preds = preds_matr[run, start_idx:end_idx, output]
                plt.plot_date(time, preds, alpha=0.3, color="gray", markersize=0, linestyle="-")
        else: 
            z = 1.96 #95% CI
            CI_low = np.subtract(pred_means,pred_stds*z)
            CI_high = np.add(pred_means,pred_stds*z)
            plt.fill_between(time,
                             CI_low[start_idx:end_idx,output], CI_high[start_idx:end_idx,output], 
                             color="gray", alpha=0.5, label="95% CI")
    
        y_pred_mean = pred_means[start_idx:end_idx,output]
        y_signal_true = y_true[start_idx:end_idx, output]
        
        plt.plot_date(time, y_pred_mean, color="darkblue", linewidth=1.5, linestyle="-", markersize=0, label="Mean prediction")
        plt.plot_date(time, y_signal_true, color="darkred", linewidth=1.5, linestyle="-", markersize=0, label="True")
        plt.ylabel(target_tags[output])
        plt.legend(frameon=True)
        if savepath is not None:
            spl = savepath.split(".")
            plt.savefig(spl[0] + "_{0}".format(output) + "." + spl[1])
        plt.show()

## Model evaluation function
This is a function that will evaluate an architecture of a neural network. It will train the model on the training data and return the necessary error metrics for the validation and training data. 

**The inputs to the function (in order):**
* `model` : A keras.models.Model() object. The architecture of the model
* `x_train` : The features of the training data
* `y_train` : The targets of the training data
* `x_valid` : The features of the validation data
* `y_valid` : The targets of the validation data
* `x_test` : The features of the test data
* `y_test` : The targets of the test data
* `name` : The name of the model
* `model_folder` : The path to the folder of the model (root folder to save plots, models etc.)
* `lookback` : The number of time steps to "look back" in the training data. Default=540. 
* `batch_size` : The number of observations in a single batch, obtained by the `batch_generator()`. Default=256. 
* `epochs` : The number of epochs to train. Default=30.
* `loss` : The loss metric to minimise. Default=`mae` - Mean Absolute Error. 
* `optimizer` : The optimizer to use when fitting the models. Default=`Adam()`
* `generator` : The batch generator. Default=`batch_generator()` defined in `main/functions.py`
* `nruns` : The number of runs that the predictions will be averaged over. Default=40,
* `makeplots` : Boolean the model should display plots or not. Default=True. 

**The outputs of the function:**
* Dictionary consisting of 
    * the trained model
    * training history 
    * validation error metrics
    * test error metrics. 

**Description:**

The evaluation function will train the model on the training data and keep track of the weights that gave the minimum error. After all training epochs it will load the weights registered on the best run. The model will then make `n` predict values for the validation and test data using the function `make_n_predictions`. The following parameters are then computed:
* `mean_preds` : The mean predictions for all features for all observations. Shape=(n_obs,n_targets).
* `mean_stds` : The standard deviation of the predictions for all features for all observations. Shape=(n_obs,n_targets)
* `expected_std` : The expected standard deviation. It is the average of `mean_stds`, i.e. the average standard deviation across all predictions for all runs. 
* `expected_mean` : The expected mean. It is the average of `mean_preds`, i.e. the average mean across all predictions for all runs. 
* `maes` : The expected MAE (averaged over `n` runs) for each target. Shape=(n_features)
* `maes_unstd` : The expected _unstandardized_ MAE (averaged over `n` runs). Shape=(n_features)
* `avg_mae` : The average of `maes`, i.e. the mean of the MAEs for all targets. 

All these parameters are calculated both for the validation set and the testing set and then collected in separate dictionaries. The model will print the performance for the validation data and testing data, before returning the trained model, training history, validation error metrics and test error metrics in a dictionary.

In [31]:
def evaluate_model(model, x_train, y_train, x_valid, y_valid, x_test, y_test, 
                   name, means, stds, root_path, feature_tags, target_tags,
                   optimizer, generator, lookback=540, batch_size=256, epochs=30, loss='mae', nruns=40, makeplots=True):
    """
    Will train the model and output model performance on validation data.
    """
    
    
    model_folder = root_path + "models/{0}/{1}/".format(name,epochs)
    if not os.path.exists(model_folder):
        print("Creating directory", model_folder)
        os.makedirs(model_folder)
    
    path_checkpoint = model_folder + "weights.keras".format(name)
    callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                          monitor='val_loss',
                                          verbose=1,
                                          save_weights_only=True,
                                          save_best_only=True)
    
    '''
    callback_reducelr = ReduceLROnPlateau(monitor = 'val_loss', 
                                          factor = 0.5,
                                          min_lr=0.001,
                                          patience = 3, 
                                          verbose = 0)
    
    #callbacks = [callback_checkpoint, callback_reducelr]
    '''
    callbacks = [callback_checkpoint]
    
    train_gen = generator(x_train, y_train, lookback, batch_size)
    model.compile(loss=loss, optimizer=optimizer)
    
    # train the model
    train_steps = int(x_train.shape[0] // batch_size)
    validation_data = (np.expand_dims(x_valid, axis=0),
                       np.expand_dims(y_valid, axis=0))
    
    history = model.fit_generator(generator=train_gen,
                                  epochs=epochs,
                                  steps_per_epoch=train_steps,
                                  validation_data=validation_data,
                                  callbacks=callbacks)
    
    # save the model 
    model.save(model_folder + "model.h5".format(name))
    
    # plot history of training and validation loss
    if makeplots:
        plot_history(history, model_folder + "history.png".format(name))
    
    # load the weights that gave the best validation error
    model.load_weights(path_checkpoint)
    
    # define some properties that we'll need
    target_stds = stds[x_train.shape[1]:]
    num_features = y_train.shape[-1]
    
    
    # == VALIDATION DATA == #
    ## Get the respecitve MAEs for each output
    preds_val, errs_val = make_n_predictions(model, x_valid, y_valid, nruns)
    mean_preds_val = np.array([np.mean(preds_val[:,:,i], axis=0) for i in range(num_features)]).T
    stds_preds_val = np.array([np.std(preds_val[:,:,i], axis=0) for i in range(num_features)]).T 
    expected_std_val = np.mean(stds_preds_val,axis=0)   # the expected standard deviation of the predictions
    expected_mean_val = np.mean(mean_preds_val,axis=0)  # the expected mean of the predictions

    mae_val = np.mean(errs_val,axis=0)    # the mean standardized validation error for each target
    mae_val_unstd = mae_val*target_stds   # the mean unstandardized validation error for each target
    avg_mae_val = np.mean(mae_val)        # the single average of all mean validation errors
    avg_mae_val_unstd = np.mean(mae_val_unstd)

    ## collect in a dataframe
    columns = ["Tag", "MAE (std)", "MAE (unstd)", "Expect. Mean", "Expect. Stdev"]
    df_val = pd.DataFrame(np.column_stack([target_tags, mae_val, mae_val_unstd, expected_mean_val, expected_std_val]), 
                          columns=columns)
    df_val.loc[len(df_val)] = ['Average', avg_mae_val, avg_mae_val_unstd, np.mean(expected_mean_val), np.mean(expected_std_val)]
    str_table_val = tabulate(df_val, headers='keys', tablefmt='psql', floatfmt='.5f')

    dict_val = {
        'df':df_val,
        'str_table':str_table_val,
        'predictions_matrix': preds_val,
        'predictions_mean': mean_preds_val,
        'predictions_stds': stds_preds_val,
    }
    print("\n       Validation data")
    print(str_table_val)
    
    
    # == TEST DATA == #
    ## Get the respecitve MAEs for each output
    preds_test, errs_test = make_n_predictions(model, x_test, y_test, nruns)
    mean_preds_test = np.array([np.mean(preds_test[:,:,i], axis=0) for i in range(num_features)]).T
    stds_preds_test = np.array([np.std(preds_test[:,:,i], axis=0) for i in range(num_features)]).T 
    expected_std_test = np.mean(stds_preds_test,axis=0)   # the expected standard deviation of the predictions
    expected_mean_test = np.mean(mean_preds_test,axis=0)  # the expected mean of the predictions

    mae_test = np.mean(errs_test,axis=0)    # the mean standardized validation error for each target
    mae_test_unstd = mae_test*target_stds   # the mean unstandardized validation error for each target
    avg_mae_test = np.mean(mae_test)         # the single average of all mean validation errors
    avg_mae_test_unstd = np.mean(mae_test_unstd)

    ## collect in a dataframe
    columns = ["Tag", "MAE (std)", "MAE (unstd)", "Expect. Mean", "Expect. Stdev"]
    df_test = pd.DataFrame(np.column_stack([target_tags, mae_test, mae_test_unstd, expected_mean_test, expected_std_test]), 
                          columns=columns)
    df_test.loc[len(df_test)] = ['Average', avg_mae_test, avg_mae_test_unstd, np.mean(expected_mean_test), np.mean(expected_std_test)]
    str_table_test = tabulate(df_test, headers='keys', tablefmt='psql', floatfmt='.5f')
    
    dict_test = {
        'df':df_test,
        'str_table':str_table_test,
        'predictions_matrix': preds_test,
        'predictions_mean': mean_preds_test,
        'predictions_stds': stds_preds_test,
    }
    print("\n       Testing data")
    print(str_table_test)
    
    # Save files
    np.save(model_folder + "targets_valid.npy", y_valid)
    np.save(model_folder + "targets_test.npy", y_test)
    np.save(model_folder + "predictions_valid.npy", preds_val)
    np.save(model_folder + "predictions_test.npy", preds_test)
    df_val.to_pickle(model_folder + "dfval.pkl")
    df_test.to_pickle(model_folder + "dftest.pkl")
    np.save(model_folder + "table_valid.npy", str_table_val)
    np.save(model_folder + "table_test.npy", str_table_test)

    # If makeplots = True, then plot the predictions with the mean 
    if makeplots: 
        plot_multiple_predictions(preds_val, y_valid, plotCI=True, 
                                  savepath=model_folder+"{0}_uncertainty_plots.png".format(name))
    
    return_dict = {
        'model': model,
        'history': history,
        'validation': dict_val,
        'test': dict_test,
    }
    
    return return_dict


In [32]:
def make_summary_for_dict(model_dict, y_valid, time_valid, y_test, time_test, target_tags, start_idx=2000, n_obs=6*60):
    model_dict['model'].summary()

    print("\n      Validation data")
    print(model_dict['validation']['str_table'])

    print("\n      Testing data")
    print(model_dict['test']['str_table'])
    
    print("\n Some sample plots:")
    print("---- Training vs validation history ----")
    plot_history(model_dict['history'])
    
    print("---- Plots of the predictions vs true ----")
    print("Validation data")
    plot_multiple_predictions(model_dict['validation']['predictions_matrix'],
                             y_valid, time_valid, target_tags, start_idx,n_obs, plotCI=False)
    
    print("Test data")
    plot_multiple_predictions(model_dict['test']['predictions_matrix'],
                         y_test, time_test, target_tags, start_idx,n_obs, plotCI=False)
    
    print("--- Plots with CI intervals ---")
    print("Validation data")
    plot_multiple_predictions(model_dict['validation']['predictions_matrix'],
                             y_valid, time_valid, target_tags, start_idx,n_obs, plotCI=True)
    
    print("Test data")
    plot_multiple_predictions(model_dict['test']['predictions_matrix'],
                         y_test, time_test, target_tags, start_idx,n_obs, plotCI=True)

In [33]:
def get_df_from_dicts(dicts, columns, index, texpath=None, round_digits=4):
    """
    Will make a dataframe with error metrics for each target tag out of a collection of dictionaries 
    as obtained by evaluate_model(). The model names will be collected as indexes in the dataframe, and the 
    target errors in the columns. 
    """
    
    val_maes = []
    test_maes = []
    for d in dicts:
        tmp_mae_val = [round(float(digit),round_digits) for digit in d['validation']['df']['MAE (std)'].tolist()]
        tmp_mae_test = [round(float(digit),round_digits) for digit in d['test']['df']['MAE (std)'].tolist()]

        val_maes.append(tmp_mae_val)
        test_maes.append(tmp_mae_test)

    # make df
    df_val = pd.DataFrame(np.vstack(val_maes), index=index, columns=columns)
    df_test = pd.DataFrame(np.vstack(test_maes), index=index, columns=columns)
    df_summary = pd.concat([df_val, df_test], axis=1, keys=["Validation", "Test"])

    tex = df_summary.to_latex(column_format="l" + "c"*(len(columns)*2),
                              multicolumn=True, 
                              multicolumn_format='c', 
                              bold_rows=True)
    if texpath is not None: 
        with open(texpath) as f:
            f.write(tex)

    return df_summary, tex

In [34]:
def get_uncertainty_df_from_dicts(dicts, columns, index, levels, texpath=None, round_digits=4):
    """
    Will make a dataframe with uncertainty and error metrics for each target tag out of a collection of dictionaries 
    as obtained by evaluate_model().
    """
    
    dataframes = []
    for d in dicts:
        df = d['validation']
        avg_maes = [round(float(digit),round_digits) for digit in df['df']['MAE (std)'].tolist()]
        exp_means = [round(float(digit),round_digits) for digit in df['df']['Expect. Mean'].tolist()]
        exp_stds = [round(float(digit),round_digits) for digit in df['df']['Expect. Stdev'].tolist()]
        df_1 = pd.DataFrame(np.column_stack([avg_maes, exp_means, exp_stds]), index = levels, columns = columns)

        df = d['test']
        avg_maes = [round(float(digit),round_digits) for digit in df['df']['MAE (std)'].tolist()]
        exp_means = [round(float(digit),round_digits) for digit in df['df']['Expect. Mean'].tolist()]
        exp_stds = [round(float(digit),round_digits) for digit in df['df']['Expect. Stdev'].tolist()]
        df_2 = pd.DataFrame(np.column_stack([avg_maes, exp_means, exp_stds]), index = levels, columns = columns)
        df_concat = pd.concat([df_1, df_2], axis=1, keys=["Validation", "Test"])
        dataframes.append(df_concat)
    
    summary_df = pd.concat(dataframes, axis=0, keys=index)
    
    tex = summary_df.to_latex(column_format="ll" + "c"*(len(columns)*2),
                              multicolumn=True, 
                              multicolumn_format='c',
                              multirow=True,
                              bold_rows=True)

    if texpath is not None: # save the file
        with open(texpath, 'w+') as f:
            f.write(tex)
    
    return summary_df, tex